In [1]:
import pandas as pd

In [2]:
import sys
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    path = '/content/drive/Othercomputers/내 MacBook Air/Documents/personal_llm_projects/dacon_merong/'
else:
    path = '../'

data_path = path + 'data/{}.csv'

Mounted at /content/drive


In [3]:
test_words =  pd.read_csv(data_path.format('test')).iloc[:, [-1]]
test_words


,사고원인
0,"펌프카 아웃트리거 바닥 고임목을 3단으로 보강 했음에도, 지반 침하(아웃트리거 우측..."
1,작업자의 불안전한 행동(숫돌 측면 사용) 및 보안면 미 착용
2,작업자가 작업을 위해 이동 중 전방을 주시하지 않아 발을 헛디뎌 계단에서 굴러 넘어짐
3,작업 발판 위 벽돌 잔재를 밟고 넘어짐
4,점심식사를 위한 이동시 작업자 부주의로 인한 추락사고 발생
...,...
959,약한 석재가 자연 파단하여 늑골 골절 됨
960,근로자가 작업 중 이동가능한 통로 확보 미흡
961,작업자가 믹서트럭 슈트아래로 불안전하게 이동하면서 콘크리트 통사이에 손가락이 끼임
962,깨진타일을 손으로 주워 마대자루에 넣을려고 하던 중 파편이 박힘


In [4]:
# my_answer.loc[4].item()

In [5]:
sub_samp =  pd.read_csv(data_path.format('sample_submission'))

In [6]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = list(test_words.iloc[:, 0])

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('jhgan/ko-sbert-sts')
model = AutoModel.from_pretrained('jhgan/ko-sbert-sts').to(device)

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt').to(device)

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, mean pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

In [7]:
tensor_df =  pd.DataFrame(sentence_embeddings.to('cpu').numpy())
sample_parsed =  pd.concat( [sub_samp.iloc[:, :2], tensor_df], axis = 1 )

In [8]:
sample_parsed.columns = sub_samp.columns
sample_parsed
sample_parsed.to_csv(data_path.format('self_return'), index = False)

In [ ]:
# sample_parsed.to_csv(path + '첫번째 제출.csv')